# ATLAS Model Training

**Spatial Transformation Specialist**

This notebook trains the ATLAS model, which specializes in:
- Geometric transformations (rotations, reflections, translations)
- Spatial transformer networks
- Affine transformation matrix operations
- Discrete operation support
- Geometric pattern recognition

## Setup Environment

In [ ]:
# Download pre-trained models (optional)
!cd /content/AutomataNexus_Olympus_AGI2 && python scripts/download_models.py

# Clone repository and install dependencies
!git clone https://github.com/AutomataControls/AutomataNexus_Olympus_AGI2.git /content/AutomataNexus_Olympus_AGI2
!cd /content/AutomataNexus_Olympus_AGI2 && pip install -r requirements.txt -q

In [ ]:
# Verify environment
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Start ATLAS Training

Training configuration:
- **Architecture**: Spatial Transformer Network
- **Parameters**: ~1.2M
- **Specialization**: Geometric transformations
- **Focus**: Rotation/reflection detection and precise parameter estimation

In [ ]:
# Pull latest updates before training
!cd /content/AutomataNexus_Olympus_AGI2 && git pull

# Train ATLAS model with OPTIMIZED SPECIALIZED TRAINER (PERFORMANCE FIXED!)
!cd /content/AutomataNexus_Olympus_AGI2 && python scripts/training/train_atlas_specialized.py

## Monitor Training Progress

In [ ]:
# Initialize ATLAS Iterative Training Controller
import json
import os
import datetime
from typing import Dict, List

class AtlasIterativeTrainer:
    def __init__(self, base_dir='/content/AutomataNexus_Olympus_AGI2'):
        self.base_dir = base_dir
        self.iteration_log_file = f"{base_dir}/arc_models_v4/atlas_iterations.json"
        self.current_params = self.load_default_params()
        self.iterations = self.load_iteration_history()
    
    def load_default_params(self) -> Dict:
        return {
            'learning_rate': 0.003,
            'batch_size': 256,
            'epochs_per_stage': 120,
            'spatial_weight': 2.0,
            'transformation_penalty': 0.5,
            'exact_match_bonus': 5.0,
            'gradient_accumulation_steps': 8,
            'stn_dropout': 0.1,
            'geometric_augmentation': True,
            'target_accuracy': 95.0
        }
    
    def load_iteration_history(self) -> List[Dict]:
        if os.path.exists(self.iteration_log_file):
            with open(self.iteration_log_file, 'r') as f:
                return json.load(f)
        return []
    
    def save_iteration_history(self):
        os.makedirs(os.path.dirname(self.iteration_log_file), exist_ok=True)
        with open(self.iteration_log_file, 'w') as f:
            json.dump(self.iterations, f, indent=2)
    
    def log_iteration(self, params: Dict, results: Dict):
        iteration = {
            'iteration': len(self.iterations) + 1,
            'timestamp': datetime.datetime.now().isoformat(),
            'parameters': params.copy(),
            'results': results.copy()
        }
        self.iterations.append(iteration)
        self.save_iteration_history()
    
    def get_best_iteration(self) -> Dict:
        if not self.iterations:
            return None
        return max(self.iterations, key=lambda x: x['results'].get('best_exact', 0))
    
    def suggest_next_params(self) -> Dict:
        if len(self.iterations) < 2:
            return self.current_params.copy()
        
        best = self.get_best_iteration()
        latest = self.iterations[-1]
        
        suggestions = best['parameters'].copy() if best else self.current_params.copy()
        
        if latest and best:
            latest_exact = latest['results'].get('best_exact', 0)
            best_exact = best['results'].get('best_exact', 0)
            
            if latest_exact < best_exact * 0.85:
                suggestions['learning_rate'] *= 0.6
                suggestions['spatial_weight'] = min(3.0, suggestions['spatial_weight'] * 1.3)
                suggestions['stn_dropout'] = max(0.05, suggestions['stn_dropout'] * 0.8)
            elif latest_exact > best_exact * 1.05:
                suggestions['learning_rate'] = min(0.005, suggestions['learning_rate'] * 1.1)
                suggestions['epochs_per_stage'] = min(150, suggestions['epochs_per_stage'] + 10)
        
        return suggestions
    
    def display_history(self):
        if not self.iterations:
            print("No ATLAS iterations found. Starting fresh!")
            return
        
        print("ATLAS Training History:")
        print("-" * 80)
        for i, iteration in enumerate(self.iterations):
            exact = iteration['results'].get('best_exact', 0)
            loss = iteration['results'].get('best_val_loss', float('inf'))
            lr = iteration['parameters'].get('learning_rate', 0)
            spatial_weight = iteration['parameters'].get('spatial_weight', 0)
            timestamp = iteration['timestamp'][:16]
            
            status = "BEST" if iteration == self.get_best_iteration() else ""
            print(f"{status} Iter {i+1}: {exact:.2f}% exact | Loss: {loss:.4f} | LR: {lr:.4f} | Spatial: {spatial_weight:.1f} | {timestamp}")
        
        print("-" * 80)
        best = self.get_best_iteration()
        if best:
            print(f"Best: Iteration {best['iteration']} with {best['results']['best_exact']:.2f}% exact match")

atlas_trainer = AtlasIterativeTrainer()
suggested_params = atlas_trainer.suggest_next_params()
atlas_trainer.display_history()
print("\nATLAS trainer initialized. You can now run the training cells below.")

In [ ]:
# ATLAS Iterative Training
import subprocess
import tempfile
import os

if 'atlas_trainer' not in globals():
    print("Error: Run the trainer initialization cell first!")
    raise NameError("atlas_trainer not defined")

LEARNING_RATE = suggested_params.get('learning_rate', 0.005)
BATCH_SIZE = suggested_params.get('batch_size', 512)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 100)
TRANSFORMATION_PENALTY = 0.5
EXACT_MATCH_BONUS = 5.0
GRADIENT_ACCUMULATION_STEPS = 4
USE_MEPT = True
USE_LEAP = True
USE_PRISM = True
TARGET_ACCURACY = 95.0

print(f"Starting ATLAS training iteration {len(atlas_trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Features: MEPT={USE_MEPT}, LEAP={USE_LEAP}, PRISM={USE_PRISM}")
print(f"Target accuracy: {TARGET_ACCURACY}%")

modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

import scripts.training.train_atlas as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.USE_MEPT = {USE_MEPT}
train_module.USE_LEAP = {USE_LEAP}
train_module.USE_PRISM = {USE_PRISM}

try:
    import scripts.training.stage0_exact_match_boost as boost_module
    original_inject = boost_module.inject_exact_match_training
    def inject_with_target(*args, **kwargs):
        kwargs['target_accuracy'] = {TARGET_ACCURACY}
        return original_inject(*args, **kwargs)
    boost_module.inject_exact_match_training = inject_with_target
except ImportError:
    print("Warning: stage0_exact_match_boost not available")

train_module.train_atlas()
"""

with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    print("Training timeout: 8 hours")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=28800
    )
    
    print("ATLAS training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    if os.path.exists(script_path):
        os.unlink(script_path)

## 🔧 Troubleshooting

In [ ]:
## Iterative Training & Hyperparameter Tuning

**Enhanced Training Loop with Checkpoint Resuming**

This section allows you to:
- Resume training from checkpoints
- Adjust hyperparameters between iterations
- Track performance improvements
- Get automated suggestions for next iteration

In [ ]:
# Clear GPU memory if needed
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
print(f"Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1e9:.2f} GB")

---

## Training Complete!

Your ATLAS model has been trained and saved to `/content/AutomataNexus_Olympus_AGI2/arc_models_v4/`

**ATLAS Specialization:**
- Geometric transformations
- Rotation and reflection detection
- Spatial pattern recognition

**AutomataNexus OLYMPUS AGI2** - *Where Neural Networks Meet Symbolic Logic*

## Iterative Training & Hyperparameter Tuning

**Enhanced Training Loop with Checkpoint Resuming**

This section allows you to:
- Resume training from checkpoints
- Adjust hyperparameters between iterations
- Track performance improvements
- Get automated suggestions for next iteration